In [1]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet
!pip install transformers --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os, re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

D0717 05:50:47.421495431      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0717 05:50:47.421517420      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0717 05:50:47.421520777      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0717 05:50:47.421523777      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0717 05:50:47.421526415      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0717 05:50:47.421529377      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0717 05:50:47.421532712      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0717 05:50:47.

In [3]:
# CHANGED FOR TPU 1VM:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("on TPU")
except tf.errors.NotFoundError:
    print("not on TPU")
    strategy = tf.distribute.MirroredStrategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)

on TPU
REPLICAS:  8


## Import Dataset

In [38]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [39]:
train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

## Exploratory Data Analysis (EDA)

First, let's check if there are any mislabeled tweets inside the dataset. As we cannot manually inspect all tweets one after the other, we are going to look for duplicate tweets and make sure these tweets have the same label(s). 

In [63]:
duplicates = train[train.duplicated('text')]
duplicates.text.nunique()

57

There are 57 duplicate tweets inside the training dataset. We are going to iterate through these duplicate tweets to see if these duplicate tweets have the same 

In [70]:
problematic_duplicates = []

for i in range(duplicates.text.nunique()):
    duplicate_subset = train[train.text == duplicates.text.unique()[i]]
    if len(duplicate_subset) > 1 and duplicate_subset.target.nunique() == 2:
        problematic_duplicates.append(i)
        
print(problematic_duplicates)

[1, 2, 4, 8, 18, 20, 21, 22, 27, 34, 37, 48, 56]


In [71]:
train[train.text == duplicates.text.unique()[1]]

,id,keyword,location,text,target
4019,5710,floods,NaN,Who is bringing the tornadoes and floods. Who ...,0
4013,5699,floods,NaN,Who is bringing the tornadoes and floods. Who ...,0
3985,5662,floods,NaN,Who is bringing the tornadoes and floods. Who ...,1


In [40]:
train = train.groupby('target').sample(3000)

In [42]:
train_df, val_df = np.split(train.sample(frac = 1), [int(0.8 * len(train))])
print('Number of rows inside the training dataset is: {}'.format(len(train_df)))
print('Number of rows inside the validation dataset is: {}'.format(len(val_df)))

Number of rows inside the training dataset is: 4800
Number of rows inside the validation dataset is: 1200


In [46]:
def clean_text(dataframe):
    dataframe.text.apply(lambda x: x.lower())
    dataframe.text.apply(lambda x: re.sub(r'http\S+', '', x))
    dataframe.text.apply(lambda x: re.sub(r'\W+', ' ', x))
    dataframe.text.apply(lambda x: re.sub(r'\d+', '', x))
    dataframe.text.apply(lambda x: x.lower())
    return(dataframe)

In [47]:
cleaned_train = clean_text(train_df)
cleaned_val = clean_text(val_df)

In [11]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess = hub.KerasLayer(tfhub_handle_preprocess)
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

In [13]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string)
encoder_input = bert_preprocess(text_input)
encoder_output = bert_encoder(encoder_input)

l = tf.keras.layers.Dense(100, activation = 'relu')(encoder_output['sequence_output'])
l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences = True, dropout = 0.3))(l)
l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences = True, dropout = 0.3))(l)
l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(25, return_sequences = True, dropout = 0.3))(l)
l = tf.keras.layers.Dense(25, activation = 'relu')(l)
l = tf.keras.layers.Dense(1, activation = 'sigmoid')(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [14]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [15]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                  patience = 2)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'model/best_performed_model.ckpt',
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_loss',
    verbose = 1
)

In [16]:
history = model.fit(cleaned_train.full_text,
                    cleaned_train.target,
                    validation_data = (cleaned_val.full_text, cleaned_val.target),
                    epochs = 30,
                    callbacks = [early_stopping, model_checkpoint_callback])

Epoch 1/30
150/150 [==============================] - ETA: 0s - loss: 0.6422 - accuracy: 0.6083
Epoch 1: val_loss improved from inf to 0.56664, saving model to model/best_performed_model.ckpt
150/150 [==============================] - 384s 2s/step - loss: 0.6422 - accuracy: 0.6083 - val_loss: 0.5666 - val_accuracy: 0.7135
Epoch 2/30
150/150 [==============================] - ETA: 0s - loss: 0.5705 - accuracy: 0.6828
Epoch 2: val_loss improved from 0.56664 to 0.52122, saving model to model/best_performed_model.ckpt
150/150 [==============================] - 349s 2s/step - loss: 0.5705 - accuracy: 0.6828 - val_loss: 0.5212 - val_accuracy: 0.7080
Epoch 3/30
150/150 [==============================] - ETA: 0s - loss: 0.5492 - accuracy: 0.6950
Epoch 3: val_loss did not improve from 0.52122
150/150 [==============================] - 349s 2s/step - loss: 0.5492 - accuracy: 0.6950 - val_loss: 0.5625 - val_accuracy: 0.7077
Epoch 4/30
150/150 [==============================] - ETA: 0s - loss: 0.5

## New Model using DistilBert

This time we are going to use DistilBERT. 

In [12]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [49]:
train_encodings = tokenizer(list(cleaned_train.text),
                            truncation = True, 
                            padding = True)
val_encodings = tokenizer(list(cleaned_val.text),
                          truncation=True, 
                          padding=True)

In [50]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                                    list(cleaned_train.target)))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),
                                                  list(cleaned_val.target)))

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                  patience = 2)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'model/best_performed_model.ckpt',
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_loss',
    verbose = 1
)

In [52]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english',
                                                              num_labels = 2)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

model.fit(train_dataset.shuffle(len(cleaned_train)).batch(32),
          epochs = 20,
          batch_size = 32,
          validation_data = val_dataset.shuffle(len(cleaned_val)).batch(32),
          callbacks = [early_stopping, model_checkpoint_callback])

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/20
150/150 [==============================] - ETA: 0s - loss: 0.7196 - accuracy: 0.5125
Epoch 1: val_loss improved from inf to 0.69378, saving model to model/best_performed_model.ckpt
150/150 [==============================] - 207s 1s/step - loss: 0.7196 - accuracy: 0.5125 - val_loss: 0.6938 - val_accuracy: 0.5075
Epoch 2/20
116/150 [======================>.......] - ETA: 39s - loss: 0.6941 - accuracy: 0.5156

KeyboardInterrupt: 

In [26]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_79 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
test['full_text'] = test.keyword + " " + test.location + " " + test.text
test.full_text = test.full_text.apply(lambda x: str(x))

test_encodings = tokenizer(list(cleaned_test.full_text),
                           truncation=True, 
                           padding=True)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                                   list(cleaned_test.target)))

model.predict()